# Deconstructing LTH (CIFAR-100)

https://arxiv.org/abs/1905.01067

# TODO: implement "large final same sign" criteria, "freeze_init_zero_all" actions, compare w/ vanilla LTH iterative magnitude pruning

# as of now, nothing has been edited. below is identical to vanilla pruning

In [1]:
import copy
import json
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchinfo import summary
from torchvision import transforms
from tqdm import tqdm

In [2]:
drive = None
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
path = "./"

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
path = path if drive is None else "/content/drive/MyDrive/self-learn/pruning"

In [5]:
sys.path.append(path)

In [6]:
from constants import *
from utils import (
    set_seed,
    train_data,
    val_data,
    train_loader,
    val_loader,
    fine_labels,
    invTrans,
)
from models import get_model_and_optimizer

set_seed()

Files already downloaded and verified
Files already downloaded and verified


In [7]:
MODEL_NAME = f"CNN_CIFAR_100_PRUNE_PCT_{PRUNE_PCT}"
print("Model Name:", MODEL_NAME)

Model Name: CNN_CIFAR_100_PRUNE_PCT_80


In [8]:
LOAD_MODEL_NAME = f"CNN_CIFAR_100_PRUNE_PCT_0"

# Data

In [9]:
# # expected: (BATCH_SIZE, 3, 32, 32), picture of mountain

# batch = next(iter(train_loader))
# print(batch[0].shape)
# test_idx = 42
# plt.imshow(batch[0][test_idx].permute(1,2,0))
# plt.title(f'{fine_labels[batch[1][test_idx]]}')

# Pruning utils

In [10]:
def init_params(m):
    """
    Initializes params for model `m` given Conv2d, BatchNorm1d,  BatchNorm2d, Linear layers.
    """
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, mean=1, std=0.02)
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data)
        nn.init.normal_(m.bias.data)

In [11]:
def reset_params(model, mask, init_state):
    """
    Resets surviving model parameters to initial values
    """
    step = 0
    for name, param in model.named_parameters():
        if "weight" in name:
            param.data = torch.from_numpy(
                init_state[name].cpu().numpy() * mask[step]
            ).to(param.device)
            step += 1
        if "bias" in name:
            param.data = init_state[name]

In [12]:
def generate_init_mask(model):
    """
    Generates initial mask matching the shape of model parameters.
    Returns:
        -Mask: List of length matching the number of weight layers in `m`, each of shape matching the corresponding weight tensor.
    """
    weight_params = [
        param.data.cpu().numpy()
        for (name, param) in model.named_parameters()
        if "weight" in name
    ]
    # TODO: maybe don't do this——could be unnecessarily memory intensive?
    # if so, revert back to prev method: sum of weight in name to get the length, then down below loop through actual model
    mask = [None] * len(weight_params)

    step = 0
    for param in weight_params:
        mask[step] = np.ones_like(param)
        step = step + 1
    return mask

In [13]:
def get_layer_weight_names(model):
    """
    Returns a list of weight layer names of model `m`.
    """
    layer_weight_names = []
    for name, _ in model.named_parameters():
        if "weight" in name:
            layer_weight_names.append(name.split(".weight")[0])
    return layer_weight_names

In [14]:
def get_num_weight_params_by_layer(mask):
    """
    Returns number of surviving (nonzero) weight parameters in a pruned model via its binary mask, by layer.
    """
    return [np.count_nonzero(mask[i]) for i in range(len(mask))]

In [15]:
def prune_by_percent(model, mask, pct):
    """
    Prunes `pct`% of parameters of model `m`, and modifies the pruning mask in-place as well.
    Specifically, this is done layerwise (p% of weights for each layer).
    """
    assert (
        isinstance(pct, (int, float)) and 0 <= pct and pct <= 100
    ), "`pct` must be a numeric value between 0 and 100 (inclusive)."
    step = 0
    for name, param in model.named_parameters():
        if "weight" in name:
            p_data_all = param.data.cpu().numpy()
            # flattened nonzero weights
            p_data = p_data_all[np.nonzero(p_data_all)]

            cutoff_val = np.percentile(
                np.abs(p_data), pct
            )  # percentile calculated on surviving params
            new_mask = np.where(np.abs(p_data_all) <= cutoff_val, 0, mask[step])
            param.data = torch.from_numpy(p_data_all * new_mask).to(param.device)
            mask[step] = new_mask
            step += 1

# todo to modify prune fn above

In [2]:
## large_final_same_sign:

#         init_sign = np.sign(raw_initial_weights)
#         final_sign = np.sign(raw_final_weights)
#         diff_sign_ind = np.squeeze(np.where(init_sign != final_sign))
#         cumulative_helped = np.abs(raw_final_weights)
#         cumulative_helped[diff_sign_ind] = 0 # prune out weights which changed sign during training
#         cumulative_helped = split_and_shape(cumulative_helped, shapes) # ignore the fn idc

#         for i, pp in enumerate(prune_percentiles):
#             if i % 2 == 1:
#                 mask_values[i] = current_mask[i] # idk. maybe every other layer is bias so they ignore?
#             else:
#                 mask_values[i] = get_mask_by_layer_by_weight(cumulative_helped[i], pp, current_mask[i])

# def get_mask_by_layer_by_weight(weight_one_layer, percentile, cur_mask = None):
# layer_shape = weight_one_layer.shape
# weight_one_layer = weight_one_layer.flatten()

# if cur_mask is not None:
#     cur_mask = cur_mask.flatten()
#     #if something is already masked out, set it to a high number to avoid pruning again
#     weight_one_layer[cur_mask == 0] = 9999
# else:
#     cur_mask = np.ones(weight_one_layer.shape)

# prune_num = np.int(np.sum(cur_mask) * percentile / 100)
# weight_ind = np.lexsort((np.random.random(weight_one_layer.size), weight_one_layer))[:prune_num]

# cur_mask[weight_ind] = 0

# return cur_mask.reshape(layer_shape)

In [ ]:
# # freeze_init_zero_all

#         freeze_init_value = initial_weights[:]
#         for i in range(len(initial_weights)):
#             temp = freeze_init_value[i].flatten()
#             init_temp = np.abs(initial_weights[i].flatten())
#             final_temp = np.abs(final_weights[i].flatten())
#             temp[np.squeeze(np.where(init_temp > final_temp))] = 0
# ## ^^ if weight moved toward 0, we just set it to 0
#             freeze_init_value[i] = temp.reshape(initial_weights[i].shape)
#             initial_weights[i] = freeze_init_value[i]

# ––––––––––––

In [16]:
def eval(model, val_loader, criterion, device):
    val_losses = []
    correct = 0
    model.eval()

    with torch.no_grad():
        for i, (img, label) in enumerate(val_loader):

            img, label = img.to(device), label.to(device)
            out = model(img)

            loss_eval = criterion(out, label)
            val_losses.append(loss_eval.item())

            pred = out.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()

    val_loss = np.mean(val_losses)
    val_acc = correct / ((len(val_loader) - 1) * BATCH_SIZE + label.size(0))

    return val_loss, val_acc

In [17]:
# ## archived: initial 100 epoch training
# def initial_train(model, train_loader, val_loader, optimizer, criterion, device):
#     model.train()
#     train_losses, val_losses = [], []
#     val_accuracies = []
#     for epoch in range(EPOCHS):

#         print(f"Epoch {epoch+1}/{EPOCHS}")

#         # compute val acc every epoch
#         val_loss, val_acc = eval(model, val_loader, criterion, device)
#         val_losses.append(val_loss)
#         val_accuracies.append(val_acc)
#         print(f"Val Loss: {val_loss:.3f} | Val Acc: {val_acc:.3f}")
#         model.train()

#         for step, (img, label) in enumerate(train_loader):

#             img, label = img.to(device), label.to(device)
#             optimizer.zero_grad()
#             out = model(img)
#             loss = criterion(out, label)
#             train_losses.append(loss.item()) # every step
#             loss.backward()

#             # Monitoring overall gradient norm
#             grads = [
#                     param.grad.detach().flatten()
#                     for param in model.parameters()
#                     if param.grad is not None
#                 ]
#             norm = torch.cat(grads).norm()

#             optimizer.step()

#             if step % PRINT_ITERS == 0:
#                 print(f"Step: {step}/{len(train_loader)} | Running Average Loss: {np.mean(train_losses):.3f} | Grad Norm: {norm:.2f}")

#         torch.save(
#             {
#                 "model_state_dict": model.state_dict(),
#                 "optimizer_state_dict": optimizer.state_dict(),
#             },
#             f"{path}/checkpoints/{MODEL_NAME}_EPOCH_{epoch+1}_SEED_{SEED}.pt",
#         )

#         with open(
#             f"{path}/train_logs/{MODEL_NAME}_SEED_{SEED}_train_losses.json", "w"
#         ) as f:
#             json.dump(train_losses, f)

#         with open(
#             f"{path}/train_logs/{MODEL_NAME}_SEED_{SEED}_val_losses.json", "w"
#         ) as f2:
#             json.dump(val_losses, f2)

#         with open(
#             f"{path}/train_logs/{MODEL_NAME}_SEED_{SEED}_val_accuracies.json", "w"
#         ) as f3:
#             json.dump(val_accuracies, f3)

#     return train_losses, val_losses, val_accuracies

In [18]:
# set_seed()
# model = Net().to(device)
# model.apply(init_params)

# optimizer = optim.AdamW(model.parameters(), lr=LR)
# criterion = nn.CrossEntropyLoss()

# # CPU: ~10 min/epoch, T4: ~45 sec
# train_losses, val_losses, val_accuracies = initial_train(model, train_loader, val_loader, optimizer, criterion, device)

# Pruning

In [19]:
def prune_train(model, train_loader, optimizer, criterion, model_pruned_pct, device):

    print(f"Training model with {model_pruned_pct:.2f}% pruned")
    model.train()
    train_losses, val_losses = [], []
    val_accuracies = []
    for epoch in range(EPOCHS):

        print(f"Epoch {epoch+1}/{EPOCHS}")

        for step, (img, label) in enumerate(train_loader):

            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()
            out = model(img)
            loss = criterion(out, label)
            train_losses.append(loss.item())  # every step
            loss.backward()

            # Monitoring overall gradient norm
            grads = [
                param.grad.detach().flatten()
                for param in model.parameters()
                if param.grad is not None
            ]
            norm = torch.cat(grads).norm()

            # Disallow pruned weights from receiving gradient updates
            for name, p in model.named_parameters():
                if "weight" in name:
                    p_data, p_grad = p.data.cpu().numpy(), p.grad.data.cpu().numpy()
                    p_grad = np.where(np.abs(p_data) < EPS, 0, p_grad)
                    p.grad.data = torch.from_numpy(p_grad).to(device)

            optimizer.step()

            ####### TEMP
            print("step ends after one, no evaluating val rn for debugging")
            if step == 0:
                break

            if step % PRINT_ITERS == 0:
                print(
                    f"Step: {step}/{len(train_loader)} | Running Average Loss: {np.mean(train_losses):.3f} | Grad Norm: {norm:.2f}"
                )

        # compute val acc at the end of every epoch

        ####### TEMP COMMENTED OUT ALSO

        # val_loss, val_acc = eval(model, val_loader, criterion, device)
        # val_losses.append(val_loss)
        # val_accuracies.append(val_acc)
        # print(f"Val Loss: {val_loss:.3f} | Val Acc: {val_acc:.3f}")
        # model.train()

        torch.save(
            {
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            },
            f"{path}/checkpoints/{MODEL_NAME}_CURR_PRUNE_PCT_{model_pruned_pct:.0f}_EPOCH_{epoch+1}_SEED_{SEED}.pt",
        )

        with open(
            f"{path}/train_logs/{MODEL_NAME}_CURR_PRUNE_PCT_{model_pruned_pct:.0f}_SEED_{SEED}_train_losses.json",
            "w",
        ) as f:
            json.dump(train_losses, f)

        with open(
            f"{path}/train_logs/{MODEL_NAME}_CURR_PRUNE_PCT_{model_pruned_pct:.0f}_SEED_{SEED}_val_losses.json",
            "w",
        ) as f2:
            json.dump(val_losses, f2)

        with open(
            f"{path}/train_logs/{MODEL_NAME}_CURR_PRUNE_PCT_{model_pruned_pct:.0f}_SEED_{SEED}_val_accuracies.json",
            "w",
        ) as f3:
            json.dump(val_accuracies, f3)

# Driver code

In [38]:
LOAD_EPOCH = 100

model, _ = get_model_and_optimizer()
model.load_state_dict(
    torch.load(
        f"{path}/checkpoints/{LOAD_MODEL_NAME}_EPOCH_{LOAD_EPOCH}_SEED_{SEED}.pt",
        map_location=device,
    )["model_state_dict"]
)
model.to(device)
print("Model loaded")

Model loaded


In [39]:
init_state = copy.deepcopy(model.state_dict())
mask = generate_init_mask(model)

layer_names = get_layer_weight_names(model)
init_num_weight_params_by_layer = get_num_weight_params_by_layer(mask)
init_num_weight_params = sum(init_num_weight_params_by_layer)

# cannot use Adam——momentum stats cause frozen weights to continue to receive updates
# optimizer = optim.AdamW(model.parameters(), lr=LR)
# could also use SGD ig
optimizer = optim.RMSprop(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

In [40]:
# orig_model_metrics = eval(model, val_loader, criterion, device)

df = pd.DataFrame(
    columns=["prune_iter", "pct_params", "pct_pruned", "val_loss", "val_acc"]
)

In [41]:
###### UNCOMMENT later

## populate with original model
# df.loc[0] = {"prune_iter": 0, "pct_params": 100.00, "pct_pruned": 0.00,
#              "val_loss": orig_model_metrics[0], "val_acc": orig_model_metrics[1]}
# df

In [42]:
EPOCHS = 1

PRUNE_PCT = 80
PRUNE_ITERS = 20
PRUNE_ITER_PCT = (1 - ((1 - PRUNE_PCT / 100) ** (1 / PRUNE_ITERS))) * 100
EPS = 1e-6
PRINT_ITERS = 1  # frequency to print train loss

# –––––––––

In [43]:
for prune_iter in range(PRUNE_ITERS):

    print(f"Prune Iteration {prune_iter+1}/{PRUNE_ITERS}")

    # prune
    prune_by_percent(model, mask, PRUNE_ITER_PCT)

    # reset remaining parameters to init state
    reset_params(model, mask, init_state)

    # log statistics
    num_weight_params_by_layer = get_num_weight_params_by_layer(mask)
    model_param_pct = (sum(num_weight_params_by_layer) / init_num_weight_params) * 100
    model_pruned_pct = 100 - model_param_pct
    print(
        f"Number of weight parameters: {sum(num_weight_params_by_layer)}/{init_num_weight_params} ≈ {model_param_pct:.2f}%"
    )

    ## create list of proportion of surviving weights (as strings) (e.g. ['72/734', '49/626'])
    params_props = [
        f"{num_weight_params_by_layer[i]}"
        + "/"
        + f"{init_num_weight_params_by_layer[i]}"
        for i in range(len(mask))
    ]
    print("Weight breakdown:")
    for i, name in enumerate(layer_names):
        print(f"  {name: <15}: {params_props[i]: <}")
    print()

    # TRAIN HERE, for j iterations
    ################################################ TEMP COMMENTED OUT FOR DEBUGGING
    prune_train(model, train_loader, optimizer, criterion, model_pruned_pct, device)
    print()

    # TODO: how to set j? simply train to convergence again?

    # evaluate and save statistics
    # model_metrics = eval(model, val_loader, criterion, device)
    # df.loc[prune_iter+1] = {"prune_iter": prune_iter+1, "pct_params": model_param_pct, "pct_pruned": model_pruned_pct,
    #                         "val_loss": model_metrics[0], "val_acc": model_metrics[1]}

    ## TEMP to avoid time consuming evaluation
    df.loc[prune_iter + 1] = {
        "prune_iter": prune_iter + 1,
        "pct_params": model_param_pct,
        "pct_pruned": model_pruned_pct,
        "val_loss": 0.8172,
        "val_acc": 0.487325,
    }

Prune Iteration 1/20
Number of weight parameters: 1684443/1825600 ≈ 92.27%
Weight breakdown:
  conv1          : 1594/1728
  conv2          : 68027/73728
  conv3          : 272109/294912
  conv4          : 1088438/1179648
  batchnorm2d_1  : 118/128
  batchnorm2d_2  : 472/512
  fc1            : 241875/262144
  fc2            : 11810/12800

Training model with 7.73% pruned
Epoch 1/1
step ends after one, no evaluating val rn for debugging

Prune Iteration 2/20
Number of weight parameters: 1554198/1825600 ≈ 85.13%
Weight breakdown:
  conv1          : 1470/1728
  conv2          : 62767/73728
  conv3          : 251069/294912
  conv4          : 1004280/1179648
  batchnorm2d_1  : 108/128
  batchnorm2d_2  : 435/512
  fc1            : 223173/262144
  fc2            : 10896/12800

Training model with 14.87% pruned
Epoch 1/1
step ends after one, no evaluating val rn for debugging

Prune Iteration 3/20
Number of weight parameters: 1434024/1825600 ≈ 78.55%
Weight breakdown:
  conv1          : 1356/17

KeyboardInterrupt: 

In [ ]:
df

# Plotting

### Part I: Train vs. validation loss of differently pruned model size checkpoints

### Part II: Validation accuracy across the pruned model checkpoints